In [16]:
import pandas as pd
import numpy as np

In [2]:
# dữ liệu dim_player

# đọc file có 3 hàng header
df = pd.read_csv(r"H:\ETL_Football\data_raw\fact_player_season_stats.csv", header=[0,1,2])

# in để kiểm tra chính xác các tuple column names nếu cần
print(df.columns.tolist())

# chọn cột cần thiết (copy để tránh SettingWithCopyWarning)
df_subset = df[[('Unnamed: 3_level_0', 'Unnamed: 3_level_1', 'player'),
                ('pos', 'Unnamed: 5_level_1', 'Unnamed: 5_level_2'),
                ('nation', 'Unnamed: 4_level_1', 'Unnamed: 4_level_2'),
                ('born', 'Unnamed: 7_level_1', 'Unnamed: 7_level_2')]].copy()

# đặt tên cột dễ dùng
df_subset.columns = ['player', 'pos', 'nation', 'born']

# loại trùng theo tên player, giữ bản ghi đầu tiên
df_subset = df_subset.drop_duplicates(subset='player', keep='first')

# thiết lập lại index để player_id liên tiếp
df_subset = df_subset.reset_index(drop=True)

# tạo player_id tăng dần từ 1
df_subset['player_id'] = np.arange(len(df_subset)) + 1

df_subset['born'] = df_subset['born'].astype('Int64')

# nếu muốn player_id là cột đầu
df_subset = df_subset[['player_id', 'player', 'pos', 'nation', 'born']]

# lưu ra file
df_subset.to_csv(r"H:\ETL_Football\data_processed\dim_player.csv", index=False)


[('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'league'), ('Unnamed: 1_level_0', 'Unnamed: 1_level_1', 'season'), ('Unnamed: 2_level_0', 'Unnamed: 2_level_1', 'team'), ('Unnamed: 3_level_0', 'Unnamed: 3_level_1', 'player'), ('nation', 'Unnamed: 4_level_1', 'Unnamed: 4_level_2'), ('pos', 'Unnamed: 5_level_1', 'Unnamed: 5_level_2'), ('age', 'Unnamed: 6_level_1', 'Unnamed: 6_level_2'), ('born', 'Unnamed: 7_level_1', 'Unnamed: 7_level_2'), ('Playing Time', 'MP', 'Unnamed: 8_level_2'), ('Playing Time', 'Starts', 'Unnamed: 9_level_2'), ('Playing Time', 'Min', 'Unnamed: 10_level_2'), ('Playing Time', '90s', 'Unnamed: 11_level_2'), ('Performance', 'Gls', 'Unnamed: 12_level_2'), ('Performance', 'Ast', 'Unnamed: 13_level_2'), ('Performance', 'G+A', 'Unnamed: 14_level_2'), ('Performance', 'G-PK', 'Unnamed: 15_level_2'), ('Performance', 'PK', 'Unnamed: 16_level_2'), ('Performance', 'PKatt', 'Unnamed: 17_level_2'), ('Performance', 'CrdY', 'Unnamed: 18_level_2'), ('Performance', 'CrdR', 'Unnamed: 19_

In [3]:
# dữ liệu dim_match
df = pd.read_csv(r"H:\ETL_Football\data_raw\fact_team_match.csv")


df_match = df[['game', 'date']].copy() # Dùng .copy() để tránh SettingWithCopyWarning

df_match = df_match.drop_duplicates(subset='game', keep='first')

# (Đây là bước quan trọng để đảm bảo ID tăng dần theo thời gian)
df_match['date'] = pd.to_datetime(df_match['date'])
df_match = df_match.sort_values(by='date', ascending=True)

# Gán một ID bắt đầu từ 1, tăng dần theo thứ tự đã sắp xếp
df_match['game_id'] = np.arange(len(df_match)) + 1

df_match['date'] = df_match['date'].dt.strftime('%Y-%m-%d')

df_match = df_match[['game_id', 'game', 'date']]

output_path = r"H:\ETL_Football\data_processed\dim_match.csv"
df_match.to_csv(output_path, index=False)

In [4]:
# dữ liệu dim_team
df = pd.read_csv(r"H:\ETL_Football\data_raw\dim_team.csv")
print(df.columns.tolist())

df_subset = df[['club_id', 'club_label', 'founding_year', 'venue_id']]
df_subset.columns = ['team_id', 'team_name', 'founded_year', 'stadium_id']

# =====================
# Làm sạch tên đội bóng
# =====================
# Danh sách từ cần loại bỏ
remove_words = ["F.C.", "F.C", "FC", "AFC", "A.F.C.", "A.F.C"]

def clean_team_name(name):
    for w in remove_words:
        name = name.replace(w, "")
    return name.strip()

df_subset["team_name"] = df_subset["team_name"].apply(clean_team_name)

name_map = {
    "Brighton & Hove Albion": "Brighton",
    "Manchester United": "Manchester utd",
    "Newcastle United": "Newcastle utd",
    "Sheffield United": "Sheffield utd",
    "Tottenham Hotspur": "Tottenham",
    "West Bromwich Albion": "West brom",
    "West Ham United": "West ham",
    "Wolverhampton Wanderers": "Wolves",
    "A Bournemouth": "Bournemouth",
    "Nottingham Forest": "Nott'ham forest"
}
# Thay thế tên đội
df_subset["team_name"] = df_subset["team_name"].replace(name_map)

# Xuất file
df_subset.to_csv(r"H:\ETL_football\data_processed\dim_team.csv", index=False)

['competition_label', 'club_label', 'founding_year', 'venue_label', 'city_label', 'capacity', 'club_id', 'venue_id']


C:\Users\admin\AppData\Local\Temp\ipykernel_10556\2976515992.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["team_name"] = df_subset["team_name"].apply(clean_team_name)
C:\Users\admin\AppData\Local\Temp\ipykernel_10556\2976515992.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["team_name"] = df_subset["team_name"].replace(name_map)


In [5]:
# dữ liệu Dim_stadium
df = pd.read_csv(r"H:\ETL_Football\data_raw\dim_team.csv")
print(df.columns.tolist())
df_subset=df[['venue_id',
              'venue_label',
              'capacity',]]
df_subset.columns=['stadium_id','statium_name','capacity']
df_subset.to_csv(r"H:\ETL_football\data_processed\dim_stadium.csv",index=False)

['competition_label', 'club_label', 'founding_year', 'venue_label', 'city_label', 'capacity', 'club_id', 'venue_id']


In [7]:
# Dữ liệu Fact_team_match

# ============================
# 1. ĐỌC DỮ LIỆU
# ============================
df = pd.read_csv(r"H:\ETL_Football\data_raw\fact_team_match.csv")
df_team = pd.read_csv(r'H:\ETL_football\data_processed\dim_team.csv')       # team_id, team_name
df_match = pd.read_csv(r'H:\ETL_football\data_processed\dim_match.csv')     # game_id, game
df_player = pd.read_csv(r'H:\ETL_football\data_processed\dim_player.csv')   # player_id, player


# ============================
# 2. CHUẨN HÓA CHUỖI (rất quan trọng)
# ============================
df['team'] = df['team'].astype(str).str.strip().str.lower()
df_team['team_name'] = df_team['team_name'].astype(str).str.strip().str.lower()

df['game'] = df['game'].astype(str).str.strip().str.lower()
df_match['game'] = df_match['game'].astype(str).str.strip().str.lower()

df['Captain'] = df['Captain'].astype(str).str.strip().str.lower()
df_player['player'] = df_player['player'].astype(str).str.strip().str.lower()

df['opponent'] = df['opponent'].astype(str).str.strip().str.lower()   # thêm chuẩn hóa opponent


# ============================
# 3. MAP TEAM → team_id
# ============================
df = df.merge(
    df_team[['team_id', 'team_name']].rename(columns={'team_name': 'team'}),
    on='team',
    how='left'
)

df.rename(columns={'team_id': 'team_id'}, inplace=True)   # giữ nguyên, cho rõ ràng

# ============================
# 6. MAP OPPONENT → opponent_id
# ============================
df = df.merge(
    df_team[['team_id', 'team_name']].rename(columns={'team_name': 'opponent'}),
    on='opponent',
    how='left',
    suffixes=('', '_opp')
)

df.rename(columns={'team_id_opp': 'opponent_id'}, inplace=True)


# ============================
# 4. MAP GAME → game_id
# ============================
df = df.merge(
    df_match[['game_id', 'game']],
    on='game',
    how='left'
)


# ============================
# 5. MAP CAPTAIN → captain_id
# ============================
df = df.merge(
    df_player[['player_id', 'player']],
    left_on='Captain',
    right_on='player',
    how='left'
)

df.rename(columns={'player_id': 'captain_id'}, inplace=True)
df.drop(columns=['player'], inplace=True)


# ============================
# 7. KIỂM TRA GIÁ TRỊ KHÔNG MAP ĐƯỢC
# ============================
print("Team không map:", df[df['team_id'].isna()]['team'].unique())
print("Opponent không map:", df[df['opponent_id'].isna()]['opponent'].unique())
print("Game không map:", df[df['game_id'].isna()]['game'].unique())
print("Captain không map:", df[df['captain_id'].isna()]['Captain'].unique())


# ============================
# 8. CHUẨN HÓA CỘT round
# ============================
df["round"] = df["round"].apply(lambda x: x.split()[-1].zfill(2))


# ============================
# 9. TẠO SUBSET CỘT FACT CUỐI
# ============================
df_subset = df[
    [
        'season',
        'game_id',
        'team_id',
        'opponent_id',
        'round',
        'venue',
        'result',
        'GF',
        'GA',
        'xG',
        'xGA',
        'Poss',
        'captain_id',
        'Formation',
        'Opp Formation',
    ]
]

print(df_subset.head())


# ============================
# 10. LƯU KẾT QUẢ
# ============================
df_subset.to_csv(r'H:\ETL_Football\data_processed\fact_team_match_clean.csv', index=False)


Team không map: []
Opponent không map: []
Game không map: []
Captain không map: []
   season  game_id team_id opponent_id round venue result  GF  GA   xG  xGA  \
0    2021        1   Q9617      Q18708    01  Away      W   3   0  1.9  0.1   
1    2021        9   Q9617      Q18747    02  Home      W   2   1  1.1  2.0   
2    2021       28   Q9617    Q1130849    03  Away      L   1   3  1.3  2.7   
3    2021       37   Q9617      Q19607    04  Home      W   2   1  0.6  0.2   
4    2021       41   Q9617      Q50602    05  Away      L   0   1  0.9  1.3   

   Poss  captain_id Formation Opp Formation  
0    54          22     3-4-3       4-2-3-1  
1    62          22     3-4-3         5-4-1  
2    34          22     3-4-3         4-3-3  
3    64          22     4-3-3         3-5-2  
4    42          22     4-3-3       3-1-4-2  


In [8]:
# dữ liệu fact_playermatchstats
df=pd.read_csv(r"h:\ETL_Football\data_raw\fact_player_match_stats.csv",header=[0,1,2])
print(df.columns.tolist())
df_match = pd.read_csv(r'H:\ETL_football\data_processed\dim_match.csv')     # game_id, game
df_player = pd.read_csv(r'H:\ETL_football\data_processed\dim_player.csv')   # player_id, player
df_team=pd.read_csv(r'H:\ETL_Football\data_processed\dim_team.csv')

df_subset=df[[('Unnamed: 1_level_0', 'Unnamed: 1_level_1', 'season'),
              ('Unnamed: 2_level_0', 'Unnamed: 2_level_1', 'game'),
              ('Unnamed: 3_level_0', 'Unnamed: 3_level_1', 'team'),
              ('Unnamed: 4_level_0', 'Unnamed: 4_level_1', 'player'),
              ('min', 'Unnamed: 9_level_1', 'Unnamed: 9_level_2'),
              ('Performance', 'Gls', 'Unnamed: 10_level_2'),
              ('Performance', 'Ast', 'Unnamed: 11_level_2'),
              ('Performance', 'PK', 'Unnamed: 12_level_2'),
              ('Performance', 'PKatt', 'Unnamed: 13_level_2'),
              ('Performance', 'Sh', 'Unnamed: 14_level_2'),
              ('Performance', 'SoT', 'Unnamed: 15_level_2'),
              ('Performance', 'CrdY', 'Unnamed: 16_level_2'),
              ('Performance', 'CrdR', 'Unnamed: 17_level_2'),
              ('Performance', 'Touches', 'Unnamed: 18_level_2'),
              ('Performance', 'Tkl', 'Unnamed: 19_level_2'),
              ('Performance', 'Int', 'Unnamed: 20_level_2'),
            ('Performance', 'Blocks', 'Unnamed: 21_level_2'),
            ('SCA', 'SCA', 'Unnamed: 25_level_2'),
            ('SCA', 'GCA', 'Unnamed: 26_level_2'),
            ('Passes', 'Cmp', 'Unnamed: 27_level_2'),
            ('Passes', 'Att', 'Unnamed: 28_level_2'),
            ('Passes', 'Cmp%', 'Unnamed: 29_level_2'),
            ('Passes', 'PrgP', 'Unnamed: 30_level_2'),
            ('Carries', 'Carries', 'Unnamed: 31_level_2'),
            ('Carries', 'PrgC', 'Unnamed: 32_level_2'),
            ('Take-Ons', 'Att', 'Unnamed: 33_level_2'),
            ('Take-Ons', 'Succ', 'Unnamed: 34_level_2')]]


df_subset.columns = [
    'season',
    'game',
    'team',
    'player',
    'min_played',
    'goals',
    'assists',
    'penalty_made',
    'penalty_attempted',
    'shots',
    'shots_on_target',
    'yellow_cards',
    'red_cards',
    'touches',
    'tackles',
    'interceptions',
    'blocks',
    'shot_creating_actions',
    'goal_creating_actions',
    'passes_completed',
    'passes_attempted',
    'pass_completion_percent',
    'progressive_passes',
    'carries',
    'progressive_carries',
    'take_ons_attempted',
    'take_ons_successful'
]


# tên khác form
name_map = {
    "Brighton & Hove Albion": "Brighton",
    "Manchester United": "Manchester utd",
    "Newcastle United": "Newcastle utd",
    "Sheffield United": "Sheffield utd",
    "Tottenham Hotspur": "Tottenham",
    "West Bromwich Albion": "West brom",
    "West Ham United": "West ham",
    "Wolverhampton Wanderers": "Wolves",
    "Nottingham Forest": "Nott'ham forest"
}
# Thay thế tên đội
df_subset["team"] = df_subset["team"].replace(name_map)



# Chuẩn hóa chuỗi cho game
df_subset['game'] = df_subset['game'].astype(str).str.strip().str.lower()
df_match['game'] = df_match['game'].astype(str).str.strip().str.lower()

# Map game → game_id
df_subset = df_subset.merge(
    df_match[['game_id', 'game']],
    on='game',
    how='left'
)



# Chuẩn hóa cho team
df_subset['team'] = df_subset['team'].astype(str).str.strip().str.lower()
df_team['team_name'] = df_team['team_name'].astype(str).str.strip().str.lower()

# Map team → team_id
df_subset = df_subset.merge(
    df_team[['team_id', 'team_name']],
    left_on='team',
    right_on='team_name',
    how='left'
)

# Xóa cột thừa
if 'team_name' in df_subset.columns:
    df_subset.drop(columns=['team_name'], inplace=True)



# Chuẩn hóa cho player
df_subset['player'] = df_subset['player'].astype(str).str.strip().str.lower()
df_player['player'] = df_player['player'].astype(str).str.strip().str.lower()

# Map player → player_id
df_subset = df_subset.merge(
    df_player[['player_id', 'player']],
    on='player',
    how='left'
)




print("Game không map:", df_subset[df_subset['game_id'].isna()]['game'].unique())
print("Team không map:", df_subset[df_subset['team_id'].isna()]['team'].unique())
print("Player không map:", df_subset[df_subset['player_id'].isna()]['player'].unique())


df_subset=df_subset[['season',
    'game_id',
    'team_id',
    'player_id',
    'min_played',
    'goals',
    'assists',
    'penalty_made',
    'penalty_attempted',
    'shots',
    'shots_on_target',
    'yellow_cards',
    'red_cards',
    'touches',
    'tackles',
    'interceptions',
    'blocks',
    'shot_creating_actions',
    'goal_creating_actions',
    'passes_completed',
    'passes_attempted',
    'pass_completion_percent',
    'progressive_passes',
    'carries',
    'progressive_carries',
    'take_ons_attempted',
    'take_ons_successful']]


df_subset.to_csv(r'H:\ETL_Football\data_processed\fact_player_match_clean.csv', index=False)



[('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'league'), ('Unnamed: 1_level_0', 'Unnamed: 1_level_1', 'season'), ('Unnamed: 2_level_0', 'Unnamed: 2_level_1', 'game'), ('Unnamed: 3_level_0', 'Unnamed: 3_level_1', 'team'), ('Unnamed: 4_level_0', 'Unnamed: 4_level_1', 'player'), ('jersey_number', 'Unnamed: 5_level_1', 'Unnamed: 5_level_2'), ('nation', 'Unnamed: 6_level_1', 'Unnamed: 6_level_2'), ('pos', 'Unnamed: 7_level_1', 'Unnamed: 7_level_2'), ('age', 'Unnamed: 8_level_1', 'Unnamed: 8_level_2'), ('min', 'Unnamed: 9_level_1', 'Unnamed: 9_level_2'), ('Performance', 'Gls', 'Unnamed: 10_level_2'), ('Performance', 'Ast', 'Unnamed: 11_level_2'), ('Performance', 'PK', 'Unnamed: 12_level_2'), ('Performance', 'PKatt', 'Unnamed: 13_level_2'), ('Performance', 'Sh', 'Unnamed: 14_level_2'), ('Performance', 'SoT', 'Unnamed: 15_level_2'), ('Performance', 'CrdY', 'Unnamed: 16_level_2'), ('Performance', 'CrdR', 'Unnamed: 17_level_2'), ('Performance', 'Touches', 'Unnamed: 18_level_2'), ('Performance', 

C:\Users\admin\AppData\Local\Temp\ipykernel_10556\2346519767.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["team"] = df_subset["team"].replace(name_map)
C:\Users\admin\AppData\Local\Temp\ipykernel_10556\2346519767.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['game'] = df_subset['game'].astype(str).str.strip().str.lower()


Game không map: []
Team không map: []
Player không map: []


In [ ]:
# dữ liệu fact team point
df=pd.read_csv(r'h:\ETL_football\data_raw\premier_league_last_5_seasons.csv')
df_team=pd.read_csv(r'H:\ETL_Football\data_processed\dim_team.csv')


def convert_season(season):
    # season dạng "2024/2025"
    parts = season.split("/")
    y1 = parts[0][-2:]   # lấy 2 số cuối của năm đầu
    y2 = parts[1][-2:]   # lấy 2 số cuối của năm sau
    return y1 + y2       # ghép lại thành 2425

df["Mùa giải"] = df["Mùa giải"].apply(convert_season)
df = df.rename(columns={"Mùa giải": "season_id"})


# tên khác form
name_map = {
    "ipswich": "Ipswich Town",
    "luton": "Luton Town",
   "newcastle": "Newcastle utd",
    "leeds": "Leeds United",
    "leicester": "Leicester City",
    "norwich": "Norwich City",
    "nottingham": "Nott'ham forest"
}
# Thay thế tên đội
df["Team"] = df["Team"].replace(name_map)


# Chuẩn hóa cho team
df['Team'] = df['Team'].astype(str).str.strip().str.lower()
df_team['team_name'] = df_team['team_name'].astype(str).str.strip().str.lower()

# Map team → team_id
df = df.merge(
    df_team[['team_id', 'team_name']],
    left_on='Team',
    right_on='team_name',
    how='left'
)

# Xóa cột thừa
if 'team_name' in df.columns:
    df.drop(columns=['team_name'], inplace=True)


print("Team không map:", df[df['team_id'].isna()]['Team'].unique())


# đổi kiểu dữ liệu cột rank
df["Rank"] = df["Rank"].astype(int)



df_subset=df[["season_id","Match_Category","Rank","team_id","MP","W","D","L","GF:GA","GD","Pts","Recent_Form"]]


In [36]:
# dữ liệu fact team point
df=pd.read_csv(r'h:\ETL_football\data_raw\premier_league_last_5_seasons.csv')
df_team=pd.read_csv(r'H:\ETL_Football\data_processed\dim_team.csv')


def convert_season(season):
    # season dạng "2024/2025"
    parts = season.split("/")
    y1 = parts[0][-2:]   # lấy 2 số cuối của năm đầu
    y2 = parts[1][-2:]   # lấy 2 số cuối của năm sau
    return y1 + y2       # ghép lại thành 2425

df["Mùa giải"] = df["Mùa giải"].apply(convert_season)
df = df.rename(columns={"Mùa giải": "season_id"})


# tên khác form
name_map = {
    "Ipswich": "Ipswich Town",
    "Luton": "Luton Town",
   "Newcastle": "Newcastle utd",
    "Leeds": "Leeds United",
    "Leicester": "Leicester City",
    "Norwich": "Norwich City",
    "Nottingham": "Nott'ham forest"
}
# Thay thế tên đội
df["Team"] = df["Team"].replace(name_map)


# Chuẩn hóa cho team
df['Team'] = df['Team'].astype(str).str.strip().str.lower()
df_team['team_name'] = df_team['team_name'].astype(str).str.strip().str.lower()



# Map team → team_id
df = df.merge(
    df_team[['team_id', 'team_name']],
    left_on='Team',
    right_on='team_name',
    how='left'
)

# Xóa cột thừa
if 'team_name' in df.columns:
    df.drop(columns=['team_name'], inplace=True)


print("Team không map:", df[df['team_id'].isna()]['Team'].unique())


# đổi kiểu dữ liệu cột rank
df["Rank"] = df["Rank"].astype(int)

# Tách GF và GA từ cột "GF:GA"
df[["GF", "GA"]] = df["GF:GA"].str.split(":", expand=True)

# Chuyển kiểu dữ liệu sang int
df["GF"] = df["GF"].astype(int)
df["GA"] = df["GA"].astype(int)

# Xóa cột cũ
df.drop(columns=["GF:GA"], inplace=True)


df_subset=df[["season_id","Match_Category","Rank","team_id","MP","W","D","L","GF","GA","GD","Pts","Recent_Form"]]

df_subset.to_csv(r'H:\ETL_Football\data_processed\fact_team_point.csv', index=False)



Team không map: []
